<a href="https://colab.research.google.com/github/z-gard/analysis/blob/main/notebooks/population_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 人口マスタ

In [1]:
!pip install -q geocoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
DIR_NAME = '/content/drive/MyDrive/z-gard/data'

In [4]:
import os
import requests
import json
import pandas as pd
from urllib.parse import urlparse
import geocoder

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [5]:
def get_file_from_url(url, save_path):
    res = requests.get(url)
    if res.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(res.content)
    else:
        raise Exception(f'status_code: {res.status_code} ({url})')

In [6]:
def number_to_kanji(str):
    return str.translate(str.maketrans({'１':'一', '２':'二', '３':'三', '４':'四', '５':'五', '６':'六', '７':'七', '８':'八', '９':'九'}))

# 緯度経度取得
def add_lat_lon(df_master):
    lats = []
    lons = []
    for i, row in df_master.iterrows():
        chocho = number_to_kanji(row['町丁目'])
        ret = geocoder.osm('東京都' + row['市区町村名'] + chocho, timeout=5)
        if not ret.ok:
            print(i, row['市区町村名'], chocho)
        lats.append(ret.lat)
        lons.append(ret.lng)
    df_copy = df_master.copy()
    df_copy['緯度'] = lats
    df_copy['経度'] = lons
    return df_copy

### 全区
- [平成27年国勢調査 東京都区市町村町丁別報告](https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd-15index.htm)

In [7]:
temp_dir = '/content/tmp'
os.makedirs(temp_dir, exist_ok=True)

zip_path = os.path.join(temp_dir, 'population.zip')
zip_url = 'https://www.toukei.metro.tokyo.lg.jp/kokusei/2015/kd15zv01100000.zip'
get_file_from_url(zip_url, zip_path)

In [8]:
!unzip -o -d $temp_dir $zip_path

Archive:  /content/tmp/population.zip
  inflating: /content/tmp/kd15zv01100000.csv  


In [9]:
csv_file = 'kd15zv01100000.csv'
df_all_popdata = pd.read_csv(os.path.join(temp_dir, csv_file), skiprows=9, encoding='shift-jis')
print(df_all_popdata.shape)

(19818, 161)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
df_popdata = df_all_popdata[[
    '地域ＩＤ',
    '地域階層フラグ／丁目･小字相当',
    '表側表章地域（階層別）／区市町村階層以上の地域名称',
    '表側表章地域（階層別）／町・大字相当階層の地域名称',
    '表側表章地域（階層別）／丁目・小字等階層の地域名称',
    '表側／男女の別',
    '人口総数（人）',
    '面積（平方キロメートル） ',
    '年齢（５歳階級）／0〜4歳（人）',
    '年齢（５歳階級）／5〜9歳（人）',
    '年齢（５歳階級）／10〜14歳（人）',
    '年齢（５歳階級）／15〜19歳（人）',
    '年齢（５歳階級）／20〜24歳（人）',
    '年齢（５歳階級）／25〜29歳（人）',
    '年齢（５歳階級）／30〜34歳（人）',
    '年齢（５歳階級）／35〜39歳（人）',
    '年齢（５歳階級）／40〜44歳（人）',
    '年齢（５歳階級）／45〜49歳（人）',
    '年齢（５歳階級）／50〜54歳（人）',
    '年齢（５歳階級）／55〜59歳（人）',
    '年齢（５歳階級）／60〜64歳（人）',
    '年齢（５歳階級）／65〜69歳（人）',
    '年齢（５歳階級）／70〜74歳（人）',
    '年齢（５歳階級）／75〜79歳（人）',
    '年齢（５歳階級）／80〜84歳（人）',
    '年齢（５歳階級）／85〜89歳（人）',
    '年齢（５歳階級）／90〜94歳（人）',
    '年齢（５歳階級）／95〜99歳（人）',
    '年齢（５歳階級）／100歳以上（人）',
    '年齢（５歳階級）／年齢不詳（人）',
    '平均年齢（歳）',
    '（再掲）外国人（人） ',
]]
df_popdata.columns = [
    '地域ＩＤ',
    '地域階層フラグ',
    '市区町村名',
    '町名',
    '町丁目',
    '男女',
    '人口総数（人）',
    '面積（km2）',
    '0〜4歳（人）',
    '5〜9歳（人）',
    '10〜14歳（人）',
    '15〜19歳（人）',
    '20〜24歳（人）',
    '25〜29歳（人）',
    '30〜34歳（人）',
    '35〜39歳（人）',
    '40〜44歳（人）',
    '45〜49歳（人）',
    '50〜54歳（人）',
    '55〜59歳（人）',
    '60〜64歳（人）',
    '65〜69歳（人）',
    '70〜74歳（人）',
    '75〜79歳（人）',
    '80〜84歳（人）',
    '85〜89歳（人）',
    '90〜94歳（人）',
    '95〜99歳（人）',
    '100歳以上（人）',
    '年齢不詳（人）',
    '平均年齢（歳）',
    '外国人（人）'
]
df_popdata = df_popdata.replace('X', 0).replace('-', 0)
df_popdata = df_popdata[
    (df_popdata['地域階層フラグ'] == '●') &
    (df_popdata['市区町村名'].str.endswith('区')) &
    (df_popdata['町丁目'].notna()) &
    (df_popdata['男女'] == '総数')
].drop(columns=['地域階層フラグ', '男女']).reset_index(drop=True)

print(df_popdata.shape)

(2921, 30)


In [11]:
pop_cols = [col for col in df_popdata.columns if '（人）' in col]
for col in pop_cols:
    df_popdata[col] = df_popdata[col].astype(int)

df_popdata['面積（km2）'] = df_popdata['面積（km2）'].astype(float)
df_popdata['平均年齢（歳）'] = df_popdata['平均年齢（歳）'].astype(float)

In [13]:
%%time
df_population_master = add_lat_lon(df_popdata)

41 千代田区 三崎町一丁目
585 墨田区 錦糸三丁目
1217 大田区 多摩川河川敷（上流）
1218 大田区 多摩川河川敷（下流）
1392 世田谷区 深沢七丁目
2616 葛飾区 お花茶屋一丁目
2617 葛飾区 お花茶屋二丁目
2618 葛飾区 お花茶屋三丁目
CPU times: user 46.7 s, sys: 3.17 s, total: 49.9 s
Wall time: 24min 32s


In [14]:
df_population_master.to_csv(os.path.join(DIR_NAME, 'population_master.csv'), index=False)

In [15]:
df_population_master[
    (df_population_master['市区町村名'] == '港区') & 
    (df_population_master['町名'] == '三田')
]

,地域ＩＤ,市区町村名,町名,町丁目,人口総数（人）,面積（km2）,0〜4歳（人）,5〜9歳（人）,10〜14歳（人）,15〜19歳（人）,20〜24歳（人）,25〜29歳（人）,30〜34歳（人）,35〜39歳（人）,40〜44歳（人）,45〜49歳（人）,50〜54歳（人）,55〜59歳（人）,60〜64歳（人）,65〜69歳（人）,70〜74歳（人）,75〜79歳（人）,80〜84歳（人）,85〜89歳（人）,90〜94歳（人）,95〜99歳（人）,100歳以上（人）,年齢不詳（人）,平均年齢（歳）,外国人（人）,緯度,経度
189,103027,港区,三田,三田１丁目,4248,0.22,239,157,129,128,167,306,405,464,470,449,332,197,150,173,161,132,113,45,24,5,1,1,41.730280,276,35.653750,139.741676
190,103028,港区,三田,三田２丁目,4025,0.30,204,140,131,138,145,256,335,360,405,350,293,240,200,262,191,153,112,72,25,8,1,4,44.224944,203,35.649845,139.741681
191,103029,港区,三田,三田３丁目,1534,0.16,65,43,32,67,105,127,131,160,143,108,109,86,68,88,67,47,38,34,9,4,1,2,43.159269,54,35.643955,139.743205
192,103030,港区,三田,三田４丁目,3400,0.24,165,113,110,111,125,162,255,319,331,294,234,207,181,253,185,146,100,64,28,9,3,5,45.651694,96,35.643942,139.739434
193,103031,港区,三田,三田５丁目,5296,0.11,227,203,193,192,215,334,437,537,518,517,419,308,275,303,204,157,129,86,35,6,1,0,43.254532,173,35.646272,139.738259
